In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import BaseCrossValidator
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.ensemble import RandomForestRegressor, \
    GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

from scipy.stats import spearmanr

In [3]:
dataset_path = '../data/model_data.h5'

# Load Data

In [4]:
data = pd.read_hdf(dataset_path, 'no_dummies')

In [5]:
data = data.drop([c for c in data.columns if 'lag' in c], axis=1)
data

,Close,Volume,Open,High,Low,Consumption in mcf,Storage in mcf,US Gross Withdrawal in mcf,Other Gross Withdrawal in mcf,RSI,...,return_21d,return_42d,return_63d,target_1d,target_5d,target_10d,target_21d,year,month,weekday
Date,,,,,,,,,,,,,,,,,,,,,
2012-04-09,2.107,108772.0,2.103,2.117,2.065,1953071.0,2478.0,2417498.0,93400.0,34.901425,...,-0.003584,-0.003797,-0.005916,-0.036070,-0.015267,-0.006449,0.007501,2012,4,0
2012-04-10,2.031,120126.0,2.111,2.125,2.025,1953071.0,2478.0,2417498.0,93400.0,30.556017,...,-0.006397,-0.004436,-0.006230,-0.023634,-0.008005,0.001807,0.009692,2012,4,1
2012-04-12,1.983,188668.0,1.976,2.068,1.972,1953071.0,2478.0,2417498.0,93400.0,28.170441,...,-0.006395,-0.005282,-0.006237,-0.001009,-0.007785,0.002641,0.011266,2012,4,3
2012-04-13,1.981,111947.0,1.982,1.999,1.959,1953071.0,2503.0,2417498.0,93400.0,28.072097,...,-0.007064,-0.005306,-0.005330,0.017668,-0.005512,0.009896,0.009795,2012,4,4
2012-04-16,2.016,115321.0,1.986,2.030,1.977,1953071.0,2503.0,2417498.0,93400.0,32.512260,...,-0.005926,-0.004447,-0.004609,-0.032242,-0.000894,0.012604,0.010299,2012,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-09,4.060,160132.0,4.168,4.185,4.002,2218011.0,2776.0,3396062.0,32500.0,61.865034,...,0.004769,0.006221,0.005180,0.007143,-0.005680,-0.002869,0.009132,2021,8,0
2021-08-10,4.089,134886.0,4.039,4.126,4.016,2218011.0,2776.0,3396062.0,32500.0,63.184390,...,0.004142,0.006239,0.005169,-0.007337,-0.012641,-0.004823,0.009921,2021,8,1
2021-08-11,4.059,139141.0,4.110,4.129,3.979,2218011.0,2776.0,3396062.0,32500.0,60.839449,...,0.004471,0.004970,0.004976,-0.031042,-0.010414,-0.004065,0.009378,2021,8,2


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2340 entries, 2012-04-09 to 2021-08-13
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Close                          2340 non-null   float64
 1   Volume                         2340 non-null   float64
 2   Open                           2340 non-null   float64
 3   High                           2340 non-null   float64
 4   Low                            2340 non-null   float64
 5   Consumption in mcf             2340 non-null   float64
 6   Storage in mcf                 2340 non-null   float64
 7   US Gross Withdrawal in mcf     2340 non-null   float64
 8   Other Gross Withdrawal in mcf  2340 non-null   float64
 9   RSI                            2340 non-null   float64
 10  ATR                            2340 non-null   float64
 11  MACD                           2340 non-null   float64
 12  return_1d                     

In [7]:
columns_to_drop = ['Open', 'Close', 'Low', 'High', 'Volume']
y = data.filter(like='target')
X = data.drop(columns_to_drop, axis=1)
X = X.drop(y.columns, axis=1)

In [8]:

class MultipleTimeSeriesCV(BaseCrossValidator):
    """Generates tuples of train_idx, test_idx pairs"""

    def __init__(self,
                 n_splits=3,
                 train_period_length=126,
                 test_period_length=21,
                 lookahead=None,
                 shuffle=False):
        self.n_splits = n_splits
        self.lookahead = lookahead
        self.test_length = test_period_length
        self.train_length = train_period_length
        self.shuffle = shuffle

    def split(self, X, y=None, groups=None):
        unique_dates = X.index.get_level_values('Date').unique()
        days = sorted(unique_dates, reverse=True)

        split_idx = []
        for i in range(self.n_splits):
            test_end_idx = i * self.test_length
            test_start_idx = test_end_idx + self.test_length
            train_end_idx = test_start_idx + self.lookahead - 1
            train_start_idx = train_end_idx + self.train_length + self.lookahead - 1
            split_idx.append([train_start_idx, train_end_idx,
                              test_start_idx, test_end_idx])

        dates = X.reset_index()[['Date']]
        for train_start, train_end, test_start, test_end in split_idx:
            train_idx = dates[(dates.Date > days[train_start])
                              & (dates.Date <= days[train_end])].index
            test_idx = dates[(dates.Date > days[test_start])
                             & (dates.Date <= days[test_end])].index
            if self.shuffle:
                np.random.shuffle(list(train_idx))
            yield train_idx, test_idx

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

In [9]:
n_splits = 20
train_period_length = 300
test_period_length = 100
lookahead = 5

cv = MultipleTimeSeriesCV(n_splits=n_splits,
                          train_period_length=train_period_length,
                          test_period_length=test_period_length,
                          lookahead=lookahead)

In [10]:
# Utilities functions

def display_score(scores):
    print('scores: ',scores)
    print('mean: ', scores.mean())
    print('standard deviation: ', scores.std())


def rank_correl(y, y_pred):
    return spearmanr(y, y_pred, axis=None)[0]

ic = make_scorer(rank_correl)


def get_cross_val_score(model, X, y, score_fun, cv, n_jobs=-1):
    cv_score = cross_val_score(estimator=model,
                           X=X,
                           y=y,
                           scoring=score_fun,
                           cv=cv,
                           n_jobs=n_jobs,
                           verbose=1)
    display_score(cv_score)

# Decision Tree Regressor

In [11]:
dt_reg = DecisionTreeRegressor(max_depth=None,
                               min_samples_split=2,
                               min_samples_leaf=1,
                               max_features='auto')


In [12]:
get_cross_val_score(dt_reg, X, y, ic, cv)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


scores:  [ 0.04945386  0.06922858  0.10260668 -0.02274714 -0.10350883  0.19043198
  0.24136651 -0.16874926 -0.00029622  0.06501219  0.02248624  0.02762176
 -0.06621727 -0.03869023  0.16765977  0.04317653  0.05114479  0.1122989
 -0.00996545  0.09372931]
mean:  0.04130213586204576
standard deviation:  0.0957447908390832


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    2.2s finished


# Random Forest Regressor

In [13]:
rf_reg = RandomForestRegressor(n_estimators=100,
                                max_depth=None,
                                min_samples_split=2,
                                min_samples_leaf=1,
                                min_weight_fraction_leaf=0.0,
                                max_features='auto',
                                max_leaf_nodes=None,
                                min_impurity_decrease=0.0,
                                min_impurity_split=None,
                                bootstrap=True,
                                oob_score=False,
                                n_jobs=-1,
                                random_state=None,
                                verbose=0,
                                warm_start=False)

In [14]:
get_cross_val_score(rf_reg, X, y, ic, cv)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


scores:  [ 0.26050363  0.1001065   0.21865655 -0.01911931 -0.0704185   0.35761388
  0.26097393 -0.1583138   0.05188917  0.21869105  0.19336682  0.07823093
  0.01551891  0.13242908  0.01696342  0.18583638  0.13980331  0.10870999
  0.10625579 -0.00318846]
mean:  0.10972546381814947
standard deviation:  0.12259217209336111


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    1.2s finished


# Ada Boosting Regressor

In [15]:
ada_reg = AdaBoostRegressor(n_estimators=100,
                            loss='square')

In [16]:
get_cross_val_score(ada_reg, X, y['target_5d'], ic, cv)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


scores:  [ 0.41809751  0.19053621  0.20933745 -0.02791291 -0.26182613  0.02117757
  0.20257581 -0.06498638  0.00078027 -0.00064294  0.07407119  0.19301152
  0.10642377  0.14595822 -0.01123086 -0.04128952 -0.02181174  0.11106277
  0.11140458 -0.08084804]
mean:  0.06369441652990833
standard deviation:  0.1410306285025524


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.7s finished


# Gradient Boosting Regressor

In [17]:
grad_reg = GradientBoostingRegressor(n_estimators=250,
                                        max_depth=None,
                                        min_samples_split=2,
                                        min_samples_leaf=1,
                                        min_weight_fraction_leaf=0.0,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        random_state=None,
                                        verbose=0,
                                        warm_start=False)

In [18]:
get_cross_val_score(grad_reg, X, y['target_5d'], ic, cv, n_jobs=1)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


scores:  [ 0.26387797  0.02144634  0.09146742 -0.14415717 -0.34500934  0.46713068
 -0.00763367 -0.19373344 -0.03123912  0.14628441  0.15767002  0.18410069
 -0.31025574  0.08784931  0.04192707  0.11481293  0.24614009  0.17808195
  0.29861503 -0.24309996]
mean:  0.05121377358974884
standard deviation:  0.20763262026519355


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   15.2s finished


It seems like that the best learner is the `RandomTreesRegressor`. Let's fine tune it in order to find
the bests parameters

## Hyperparamter Options

In [19]:
n_estimators = [100, 250]
max_depth = [5, 15, None]
min_samples_leaf = [5, 25, 100]

In [20]:
from itertools import product

cv_params = list(product(n_estimators,
                         max_depth,
                         min_samples_leaf))
n_cv_params = len(cv_params)
n_cv_params

18

In [21]:
sample_proportion = .6
sample_size = int(sample_proportion * n_cv_params)

cv_param_sample = np.random.choice(list(range(n_cv_params)),
                                     size=int(sample_size),
                                     replace=False)
cv_params_ = [cv_params[i] for i in cv_param_sample]
print('# CV parameters:', len(cv_params_))

# CV parameters: 10


## Train/Test Period Lenghts

In [22]:
YEAR = 252
train_lengths = [5 * YEAR, 3 * YEAR, YEAR, 126, 63]
test_lengths = [5, 21]

In [23]:
test_params = list(product(train_lengths, test_lengths))
n_test_params = len(test_params)
print('# period params: ', n_test_params)

# period params:  10


## Run cross-validation

In [24]:
labels = sorted(y.columns)
features = X.columns.tolist()
lookaheads = [1, 5, 10, 21]

In [25]:
label_dict = dict(zip(lookaheads, labels))

In [26]:
cross_val_cols = [
    'train_length',
    'test_length',
    'n_estimators',
    'max_depth',
    'min_samples_leaf',
    'mean_ic',
    'std_ic',
    'rounds',
    'target'
]

cross_val_results = pd.DataFrame(columns=cross_val_cols)

In [ ]:
for lookahead in lookaheads:
    for train_length, test_length in test_params:
        n_splits = int(4 * YEAR / test_length)
        res_row = dict(zip(cross_val_results, [None] * len(cross_val_results)))

        cv = MultipleTimeSeriesCV(n_splits=n_splits,
                                  test_period_length=test_length,
                                  train_period_length=train_length,
                                  lookahead=lookahead)

        res_row['target'] = lookahead
        res_row['train_length'] = train_length
        res_row['test_length'] = test_length


        for p, (n_estimator, max_d, min_s_l) in enumerate(cv_params_):
            base_params = rf_reg.get_params()
            model = RandomForestRegressor(base_params)
            model_params = {
                'n_estimators': n_estimator,
                'max_depth': max_d,
                'max_features': 'log2',
                'min_samples_leaf': min_s_l
            }
            model = model.set_params(**model_params)

            cval_score = cross_val_score(
                estimator=model,
                X=X,
                y=y,
                cv=cv)

            for k, v in model_params.items():
                res_row[k] = v

            res_row['mean_ic'] = cval_score.mean()
            res_row['std_ic'] = cval_score.std()
            res_row['rounds'] = len(cval_score)
            cross_val_results = cross_val_results.append(res_row, ignore_index=True)

KeyboardInterrupt: 

In [28]:
cross_val_results.head()

,train_length,test_length,n_estimators,max_depth,min_samples_leaf,mean_ic,std_ic,rounds,target,max_features
0,1260,5,100,15,100,-12.468692,27.517128,201,1,log2
1,1260,5,250,5,25,-11.867584,25.711685,201,1,log2
2,1260,5,250,15,100,-12.503860,27.633844,201,1,log2
3,1260,5,100,None,25,-12.151129,26.922979,201,1,log2
4,1260,5,100,5,100,-12.547710,27.510026,201,1,log2


In [29]:
save =True

if save:
    cross_val_results.to_hdf('./rf_cross_val_results.h5', 'res', mode='w')


In [30]:
bla bla bla

SyntaxError: invalid syntax (Temp/ipykernel_19100/521198524.py, line 1)

In [ ]:

from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [50, 100, 250],
              'max_depth': [5, 15, None],
              'min_samples_leaf': [5, 25, 100],
              'max_features': ['auto', 'sqrt', 'log2']}

gridsearch_reg = GridSearchCV(estimator=rf_reg,
                              param_grid=param_grid,
                              scoring=ic,
                              n_jobs=-1,
                              cv=cv,
                              refit=True,
                              return_train_score=True,
                              verbose=1)

In [ ]:
gridsearch_reg.fit(X, y.target_5d.ravel())

In [ ]:
best_learner = gridsearch_reg.best_estimator_

In [ ]:
import pickle

best_model_filename = './best_tree_model.pkl'

with open(best_model_filename, 'wb') as file:
    pickle.dump(best_learner, file)


In [ ]:
gridsearch_reg.best_params_

In [ ]:
f'{gridsearch_reg.best_score_:.2f}'

## Evaluate the Best Model

In [ ]:
for predicted, actual in zip(best_learner.predict(X.iloc[-15:]), y.target_5d.values[-15:]):
    print('Predicted: ', predicted)
    print('Actual: ', actual)
    print('Spread: ', np.abs(predicted - actual), end='\n\n')

In [ ]:
print(rank_correl(best_learner.predict(X.iloc[-15:]), y.target_5d.values[-15:]))


## Parameters importance

In [ ]:

fig, ax = plt.subplots(figsize=(12,5))
(pd.Series(best_learner.feature_importances_, index=X.columns)
 .sort_values(ascending=False)
 .iloc[:20]
 .sort_values()
 .plot.barh(ax=ax, title='Feature Importance'))

sns.despine()

fig.tight_layout();
